# Setup:


In [ ]:
!pip install torchinfo
!pip install torch
!pip install torchvision
!pip install tqdm
!pip install matplotlib
!pip install numba

In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
import random
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import tqdm
import os
from pathlib import Path
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from torchvision import models
import torch.optim as optim
import numpy as np
from PIL import Image
from google.colab import drive
import torchinfo
from torchinfo import summary

device = "cuda" if torch.cuda.is_available() else "cpu"
device

Path to data from drive

In [ ]:
drive.mount('/content/drive')
image_path = Path("Path")
train_path = image_path / "train"
train_path

Transformation settings

In [ ]:
data_transform = transforms.Compose([
    transforms.RandomRotation((0,10)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomResizedCrop(size=(64, 64), scale=(0.9, 1.0)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_path,
                                  transform=data_transform,
                                  target_transform=None)

Sample image with transformations

In [ ]:
i = 0
image, label = train_data[i][0], train_data[i][1]

image_array = image.permute(1, 2, 0).cpu().numpy()

plt.imshow(image_array)
plt.axis('off')
plt.show()

Batch setting

In [ ]:
BATCH_SIZE = 64

train_dataloader = DataLoader(train_data,
    batch_size=BATCH_SIZE,
    num_workers=2,
    shuffle=True
)

Models setup

In [ ]:
class Generator(nn.Module):
  def __init__(self,latent_space_size = 100,
               ngf = 64,
               n_channel = 3):

    super(Generator,self).__init__()

    self.generator = nn.Sequential(
            nn.ConvTranspose2d( latent_space_size, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d( ngf, n_channel, 4, 2, 1, bias=False),
            nn.Tanh()
        )

  def forward(self,inp):
    return self.generator(inp)

G = Generator()
G
x = torch.randn((1, 100, 1, 1))
G(x).shape

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,n_channel = 3,
               ndf = 64):
    super(Discriminator,self).__init__()

    self.discriminator = nn.Sequential(

            nn.Conv2d(n_channel, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

  @staticmethod
  def linear_block(in_ftrs,out_ftrs,p):
    return nn.Sequential(
        nn.Linear(in_ftrs,out_ftrs),
        nn.BatchNorm1d(out_ftrs),
        nn.ReLU(),
        nn.Dropout(p)
    )


  def forward(self,inp):
    return self.discriminator(inp)


D = Discriminator()
D(G(x)).shape

Here you can load a trained models

In [ ]:
LOAD_PATH_D = Path(r"discriminator_path")
LOAD_PATH_G = Path(r"generator_path")

G.load_state_dict(torch.load(LOAD_PATH_G))
D.load_state_dict(torch.load(LOAD_PATH_D))

# Training

In [ ]:
G.to(device)
D.to(device)

LEARNING_RATE = 0.0001
BATCH_SIZE = 64
epochs = 10000
fixed_noise = torch.randn((1, 100, 1, 1))

opt_generator = optim.Adam(G.parameters(), lr=LEARNING_RATE)
opt_discriminator = optim.Adam(D.parameters(), lr=LEARNING_RATE)
loss_fn = nn.BCELoss()

G.train()
D.train()


for epoch in tqdm(range(epochs)):
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        real = X
        noise = torch.randn(BATCH_SIZE, 100, 1, 1).to(device)
        fake = G(noise)


        disc_real = D(real).reshape(-1)
        loss_disc_real = loss_fn(disc_real, torch.ones_like(disc_real))
        disc_fake = D(fake.detach()).reshape(-1)
        loss_disc_fake = loss_fn(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        D.zero_grad()
        loss_disc.backward()
        opt_discriminator.step()

        output = D(fake).reshape(-1)
        loss_gen = loss_fn(output, torch.ones_like(output))
        G.zero_grad()
        loss_gen.backward()
        opt_generator.step()


# Testing

Ploting a sample of the Generator output

In [ ]:
G.to("cpu")
G.eval()
noise = torch.randn(1, 100, 1, 1)
image = G(noise)
image = image.squeeze()

image_array = image.permute(1, 2, 0).detach().numpy()
plt.figure(figsize=(10, 10))
plt.imshow(image_array)
plt.axis('off')
plt.show()

print(image.shape)